In [1]:
"""
download_fred_data.py

Script to automatically download all required macro/financial time series
from FRED (and a few related sources where applicable) and save them as CSVs.

No FRED API key required – uses the public CSV download endpoint.
"""

import os
import time
import requests
import pandas as pd


BASE_DIR = os.getcwd()      # <-- works inside Jupyter
DATA_DIR = os.path.join(BASE_DIR, "raw")

os.makedirs(DATA_DIR, exist_ok=True)


FRED_SERIES = {
    # --- Yield Curve: Treasuries ---
    "DGS3MO": "3-Month Treasury Constant Maturity Rate",
    "DGS1": "1-Year Treasury Constant Maturity Rate",
    "DGS2": "2-Year Treasury Constant Maturity Rate",
    "DGS5": "5-Year Treasury Constant Maturity Rate",
    "DGS7": "7-Year Treasury Constant Maturity Rate",
    "DGS10": "10-Year Treasury Constant Maturity Rate",
    "DGS20": "20-Year Treasury Constant Maturity Rate",
    "DGS30": "30-Year Treasury Constant Maturity Rate",

    # --- Recession Indicators ---
    "USREC": "US Recession Indicator (monthly)",
    "USRECM": "US Recession Indicator (alt monthly series)",

    # --- CPI (Inflation) ---
    "CPIAUCSL": "CPI All Urban Consumers (Headline)",
    "CPILFESL": "CPI All Urban Consumers: Core (Ex Food & Energy)",
    "CPIENGSL": "CPI Energy",
    "CPIFABSL": "CPI Food and Beverages",
    "CPIGODSL": "CPI Commodities (Goods)",
    "CPISRVSL": "CPI Services",

    # --- PCE Price Index (Fed’s preferred) ---
    "PCEPI": "PCE Price Index (Headline)",
    "PCEPILFE": "PCE Price Index (Core, Ex Food & Energy)",
    "PCEPIS": "PCE Services",
    "PCEPISDG": "PCE Durable Goods",

    # --- PPI (Producer Prices) ---
    "PPIFGS": "PPI: Final Demand",
    "PPIENG": "PPI: Energy",
    "PPICMM": "PPI: Commodities",

    # --- Supply/Demand Shock Proxies ---
    "DCOILWTICO": "WTI Crude Oil Spot Price",
    "PALLFNFINDEX": "Global Price Index of All Commodities",

    # NOTE: GSCPI is also on FRED now
    "GSCPI": "Global Supply Chain Pressure Index",

    # --- Unemployment: Headline ---
    "UNRATE": "Unemployment Rate (Total, 16+)",

    # --- Unemployment by Gender ---
    "LNS14000001": "Unemployment Rate - Men, 16+",
    "LNS14000002": "Unemployment Rate - Women, 16+",

    # --- Unemployment by Race ---
    "LNS14000003": "Unemployment Rate - White, 16+",
    "LNS14000006": "Unemployment Rate - Black or African American, 16+",
    "LNS14000009": "Unemployment Rate - Hispanic or Latino, 16+",

    # --- Unemployment by Age ---
    "LNS14000012": "Unemployment Rate - 16 to 19 years",
    "LNS14000089": "Unemployment Rate - 20 to 24 years",
    "LNS14000025": "Unemployment Rate - 25 to 54 years",
    "LNS14000036": "Unemployment Rate - 55 years and over",

    # --- Market-Based Inflation Expectations (TIPS) ---
    "T5YIE": "5-Year Breakeven Inflation Rate",
    "T10YIE": "10-Year Breakeven Inflation Rate",
    "T5YIFR": "5-Year, 5-Year Forward Inflation Expectation Rate",

    # --- Survey-Based Inflation Expectations ---
    "MICH": "University of Michigan: Inflation Expectation (12-month)",
    "MICH5YMV": "University of Michigan: Inflation Expectation (5-year)",

    # --- Wages / Labor Cost (optional but useful) ---
    "ECIALLCIV": "Employment Cost Index: Total Compensation for Civilians",
}


# ----------------------------
# 3. HELPER: DOWNLOAD FROM FRED
# ----------------------------

def download_fred_series(series_id: str, desc: str, out_dir: str = DATA_DIR,
                         sleep_seconds: float = 0.5) -> None:
    """
    Download a single FRED series as CSV using the public endpoint.

    Saves to: data/raw/<series_id>.csv
    """
    url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}"
    out_path = os.path.join(out_dir, f"{series_id}.csv")

    print(f"Downloading {series_id} - {desc} ...")

    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()

        # Save raw CSV as returned by FRED
        with open(out_path, "wb") as f:
            f.write(r.content)

        # Optional: sanity check using pandas, and re-save as clean CSV
        df = pd.read_csv(out_path)
        # Standardize column names: DATE, VALUE -> date, value
        df.columns = [c.lower() for c in df.columns]
        # Convert date column to datetime
        if "date" in df.columns:
            df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df.to_csv(out_path, index=False)

        print(f"✔ Saved to {out_path} (rows: {len(df)})")

    except Exception as e:
        print(f"✖ Failed to download {series_id}: {e}")

    # Be polite to FRED servers
    time.sleep(sleep_seconds)


# ----------------------------
# 4. MAIN: LOOP OVER ALL SERIES
# ----------------------------

def main():
    print(f"Saving data into: {DATA_DIR}")
    print(f"Total FRED series to download: {len(FRED_SERIES)}")
    print("-" * 60)

    for sid, desc in FRED_SERIES.items():
        download_fred_series(sid, desc)

    print("-" * 60)
    print("Finished downloading all series.")
    print("You can now load them from data/raw/*.csv in your EDAV notebook.")


if __name__ == "__main__":
    main()


Saving data into: /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw
Total FRED series to download: 42
------------------------------------------------------------
✔ Saved to /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw/DGS3MO.csv (rows: 11540)
✔ Saved to /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw/DGS1.csv (rows: 16670)
✔ Saved to /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw/DGS2.csv (rows: 12910)
✔ Saved to /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw/DGS5.csv (rows: 16670)
✔ Saved to /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw/DGS7.csv (rows: 14715)
✔ Saved to /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw/DGS10.csv (rows: 16670)
✔ Saved to /Users/raghav/Documents/Columbia/Fall 2025/EDAV/bendsbeforethebreak/data/raw/DGS20.csv (rows: 16670)
✔ Saved to /Users/raghav/Docu

In [2]:
import os
import pandas as pd
import numpy as np

RAW_DIR = "raw"
CLEAN_DIR = "clean"
os.makedirs(CLEAN_DIR, exist_ok=True)

# Display settings
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)


In [3]:
def load_and_clean_fred_series(path):
    """
    Load a FRED CSV and return a DataFrame with:
      - 'date' column (parsed as datetime)
      - one value column named after the series ID

    It assumes:
      - first column = date-like
      - second column = values
    """
    series_id = os.path.basename(path).split(".")[0]

    df = pd.read_csv(path)

    # Minimal sanity check
    if df.shape[1] < 2:
        raise ValueError(f"Unexpected format in {path}: need at least 2 columns, got {df.shape[1]}")

    # Take first two columns regardless of their names
    date_col = df.columns[0]
    value_col = df.columns[1]

    df = df[[date_col, value_col]].copy()

    # Rename to standard names
    df.rename(columns={date_col: "date", value_col: series_id}, inplace=True)

    # Parse dates & values
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df[series_id] = pd.to_numeric(df[series_id], errors="coerce")

    # Drop rows where date is missing
    df = df.dropna(subset=["date"])

    return df[["date", series_id]]



In [4]:
all_series = {}

for filename in os.listdir(RAW_DIR):
    if filename.endswith(".csv"):
        sid = filename.replace(".csv", "")
        path = os.path.join(RAW_DIR, filename)
        try:
            df = load_and_clean_fred_series(path)
            all_series[sid] = df
            print("Loaded:", sid, "shape:", df.shape)
        except Exception as e:
            print("Error loading", sid, ":", e)



Loaded: DGS10 shape: (16670, 2)
Loaded: DGS2 shape: (12910, 2)
Loaded: DGS1 shape: (16670, 2)
Loaded: DGS5 shape: (16670, 2)
Loaded: PCEPILFE shape: (800, 2)
Loaded: DGS7 shape: (14715, 2)
Loaded: LNS14000089 shape: (933, 2)
Loaded: USREC shape: (2051, 2)
Loaded: PPIENG shape: (1197, 2)
Loaded: MICH shape: (574, 2)
Loaded: PPICMM shape: (945, 2)
Loaded: LNS14000012 shape: (933, 2)
Loaded: LNS14000006 shape: (645, 2)
Loaded: LNS14000003 shape: (861, 2)
Loaded: LNS14000002 shape: (933, 2)
Loaded: CPIFABSL shape: (705, 2)
Loaded: LNS14000001 shape: (933, 2)
Loaded: CPILFESL shape: (825, 2)
Loaded: T10YIE shape: (5974, 2)
Loaded: UNRATE shape: (933, 2)
Loaded: LNS14000025 shape: (933, 2)
Loaded: USRECM shape: (2051, 2)
Loaded: DGS3MO shape: (11540, 2)
Loaded: T5YIFR shape: (5974, 2)
Loaded: PPIFGS shape: (825, 2)
Loaded: LNS14000036 shape: (933, 2)
Loaded: CPIAUCSL shape: (945, 2)
Loaded: LNS14000009 shape: (631, 2)
Loaded: DCOILWTICO shape: (10403, 2)
Loaded: DGS30 shape: (12725, 2)
Loade

In [5]:
master_df = None

for sid, df in all_series.items():
    if master_df is None:
        master_df = df
    else:
        master_df = master_df.merge(df, on="date", how="outer")

master_df = master_df.sort_values("date").drop_duplicates("date").reset_index(drop=True)
print("Merged shape:", master_df.shape)
master_df.head()


Merged shape: (18175, 36)


,date,DGS10,DGS2,DGS1,DGS5,PCEPILFE,DGS7,LNS14000089,USREC,PPIENG,MICH,PPICMM,LNS14000012,LNS14000006,LNS14000003,LNS14000002,CPIFABSL,LNS14000001,CPILFESL,T10YIE,UNRATE,LNS14000025,USRECM,DGS3MO,T5YIFR,PPIFGS,LNS14000036,CPIAUCSL,LNS14000009,DCOILWTICO,DGS30,T5YIE,ECIALLCIV,CPIENGSL,DGS20,PCEPI
0,1854-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1855-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1855-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1855-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1855-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Use end-of-month values for all series
master_df = master_df.set_index("date").resample("M").last()
master_df.index.name = "date"

print("After monthly resampling:", master_df.shape)
master_df.head()


After monthly resampling: (2052, 35)


/var/folders/_3/fgb1m84x6ldc3bc0lnk344xc0000gn/T/ipykernel_77083/3954277428.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  master_df = master_df.set_index("date").resample("M").last()


,DGS10,DGS2,DGS1,DGS5,PCEPILFE,DGS7,LNS14000089,USREC,PPIENG,MICH,PPICMM,LNS14000012,LNS14000006,LNS14000003,LNS14000002,CPIFABSL,LNS14000001,CPILFESL,T10YIE,UNRATE,LNS14000025,USRECM,DGS3MO,T5YIFR,PPIFGS,LNS14000036,CPIAUCSL,LNS14000009,DCOILWTICO,DGS30,T5YIE,ECIALLCIV,CPIENGSL,DGS20,PCEPI
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1854-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1855-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
master_df.notna().any(axis=1).idxmax()   # first date where at least one series has data
master_df.loc["1954":].head()           # or directly look from 1954 onwards


,DGS10,DGS2,DGS1,DGS5,PCEPILFE,DGS7,LNS14000089,USREC,PPIENG,MICH,PPICMM,LNS14000012,LNS14000006,LNS14000003,LNS14000002,CPIFABSL,LNS14000001,CPILFESL,T10YIE,UNRATE,LNS14000025,USRECM,DGS3MO,T5YIFR,PPIFGS,LNS14000036,CPIAUCSL,LNS14000009,DCOILWTICO,DGS30,T5YIE,ECIALLCIV,CPIENGSL,DGS20,PCEPI
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1954-01-31,NaN,NaN,NaN,NaN,NaN,NaN,4.3,1.0,13.5,NaN,31.0,12.1,NaN,4.5,5.9,NaN,4.4,NaN,NaN,4.9,4.0,1.0,NaN,NaN,30.5,8.7,26.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-02-28,NaN,NaN,NaN,NaN,NaN,NaN,4.8,1.0,13.5,NaN,30.3,13.5,NaN,4.9,5.9,NaN,4.9,NaN,NaN,5.2,4.4,1.0,NaN,NaN,30.4,9.2,26.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-03-31,NaN,NaN,NaN,NaN,NaN,NaN,5.4,1.0,13.3,NaN,31.0,13.0,NaN,5.0,6.4,NaN,5.3,NaN,NaN,5.7,4.8,1.0,NaN,NaN,30.4,9.2,26.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-04-30,NaN,NaN,NaN,NaN,NaN,NaN,5.5,1.0,13.2,NaN,32.1,13.6,NaN,5.5,6.4,NaN,5.6,NaN,NaN,5.9,5.1,1.0,NaN,NaN,30.6,9.8,26.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-05-31,NaN,NaN,NaN,NaN,NaN,NaN,5.9,1.0,13.2,NaN,31.8,13.4,NaN,5.3,6.3,NaN,5.7,NaN,NaN,5.9,5.2,1.0,NaN,NaN,30.6,9.4,26.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Common macro period: post-1954 (when many FRED series start)
master_df = master_df.loc["1954-01-31":]
master_df.head()


,DGS10,DGS2,DGS1,DGS5,PCEPILFE,DGS7,LNS14000089,USREC,PPIENG,MICH,PPICMM,LNS14000012,LNS14000006,LNS14000003,LNS14000002,CPIFABSL,LNS14000001,CPILFESL,T10YIE,UNRATE,LNS14000025,USRECM,DGS3MO,T5YIFR,PPIFGS,LNS14000036,CPIAUCSL,LNS14000009,DCOILWTICO,DGS30,T5YIE,ECIALLCIV,CPIENGSL,DGS20,PCEPI
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1954-01-31,NaN,NaN,NaN,NaN,NaN,NaN,4.3,1.0,13.5,NaN,31.0,12.1,NaN,4.5,5.9,NaN,4.4,NaN,NaN,4.9,4.0,1.0,NaN,NaN,30.5,8.7,26.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-02-28,NaN,NaN,NaN,NaN,NaN,NaN,4.8,1.0,13.5,NaN,30.3,13.5,NaN,4.9,5.9,NaN,4.9,NaN,NaN,5.2,4.4,1.0,NaN,NaN,30.4,9.2,26.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-03-31,NaN,NaN,NaN,NaN,NaN,NaN,5.4,1.0,13.3,NaN,31.0,13.0,NaN,5.0,6.4,NaN,5.3,NaN,NaN,5.7,4.8,1.0,NaN,NaN,30.4,9.2,26.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-04-30,NaN,NaN,NaN,NaN,NaN,NaN,5.5,1.0,13.2,NaN,32.1,13.6,NaN,5.5,6.4,NaN,5.6,NaN,NaN,5.9,5.1,1.0,NaN,NaN,30.6,9.8,26.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954-05-31,NaN,NaN,NaN,NaN,NaN,NaN,5.9,1.0,13.2,NaN,31.8,13.4,NaN,5.3,6.3,NaN,5.7,NaN,NaN,5.9,5.2,1.0,NaN,NaN,30.6,9.4,26.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
master_df = master_df.loc["1954-01-31":]


In [10]:
master_df.isna().sum().sort_values()


USRECM           1
USREC            1
CPIAUCSL         2
LNS14000036      2
LNS14000025      2
              ... 
DCOILWTICO     384
T10YIE         588
T5YIFR         588
T5YIE          588
ECIALLCIV      765
Length: 35, dtype: int64

In [11]:
threshold = 0.95
master_df = master_df.loc[:, master_df.isna().mean() < threshold]


In [12]:
yield_cols = [c for c in master_df.columns if c.startswith("DGS")]
master_df[yield_cols] = master_df[yield_cols].ffill()


In [13]:
macro_cols = [c for c in master_df.columns 
              if c not in yield_cols and c != "USREC"]
master_df[macro_cols] = master_df[macro_cols].ffill()


In [14]:
if "USREC" in master_df.columns:
    master_df["USREC"] = master_df["USREC"].fillna(0).astype(int)


In [15]:
master_df = master_df.dropna(how="all")


In [16]:
print(master_df.shape)
master_df.isna().sum().sort_values(ascending=False)


(863, 35)


T10YIE         588
T5YIE          588
T5YIFR         588
ECIALLCIV      564
DCOILWTICO     384
              ... 
LNS14000003      0
LNS14000002      0
USRECM           0
LNS14000025      0
CPIAUCSL         0
Length: 35, dtype: int64

In [17]:
master_path = os.path.join(CLEAN_DIR, "master_df.csv")
master_df.to_csv(master_path)
print("Saved cleaned dataset to:", master_path)


Saved cleaned dataset to: clean/master_df.csv
